In [ ]:
import librosa
import librosa.display
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from transformers import BertModel, BertTokenizer
import torch
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
from cca_zoo.deepmodels import DCCA
from torchvggish import vggish, vggish_input
import sys
import random
import csv
from transformers import BertForSequenceClassification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# os.environ['CUDA_VISIBLE_DEVICES'] ='0'
# model_urls = {
#     'vggish': 'https://github.com/harritaylor/torchvggish/'
#               'releases/download/v0.1/vggish-10086976.pth',
#     'pca': 'https://github.com/harritaylor/torchvggish/'
#            'releases/download/v0.1/vggish_pca_params-970ea276.pth'
# }

In [ ]:
# def VGGISH(**kwargs):
#     model = VGGish(urls=model_urls, **kwargs)
#     return model

In [ ]:
'''
dirname        : path that need to be searched
ret                : files in the dirname (recursive)
list_avoid_dir : dirname need to be skipped
usage           : 
    list_files = []
    file_search(dirname, list_files):   
'''
def file_search(dirname, ret, list_avoid_dir=[]):
    
    filenames = os.listdir(dirname)
    
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)

        if os.path.isdir(full_filename) :
            if full_filename.split('/')[-1] in list_avoid_dir:
                continue
            else:
                file_search(full_filename, ret, list_avoid_dir)
            
        else:
            ret.append(full_filename)          

            

'''
filename : filename (inc. path) that will be inspected
'''
def find_encoding(filename):
    rawdata = open(filename, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']    
    return charenc
            
'''
dir_name : dir_name (inc. path) that will be created ( full-path name )
'''
def create_folder(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [ ]:
def extract_trans(list_in_file, out_file):
    
    lines = []
    
    for in_file in list_in_file:
        cnt = 0
        
        with open(in_file, 'r') as f1:
            lines = f1.readlines()

        with open(out_file, 'a') as f2:

            csv_writer = csv.writer(f2)
            lines = sorted(lines)                  # sort based on first element
            
            for line in lines:

                name = line.split(':')[0].split(' ')[0].strip()
                
                # unwanted case 
                if name[:3] != 'Ses':             # noise transcription such as reply  M: sorry
                    continue
                elif name[-3:-1] == 'XX':        # we don't have matching pair in label
                    continue
                trans = line.split(':')[1].strip()
                
                cnt += 1
                csv_writer.writerow([name, trans])

In [ ]:
# [schema] ID, transcriptions [csv]

list_files = []

for x in range(5):
    sess_name = 'Session' + str(x+1)

    path = '/media/phamnhattruong/ReSe/Datasets/Emotion/IEMOCAP/IEMOCAP_full_release/IEMOCAP_full_release/' + sess_name + '/dialog/transcriptions/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print (sess_name + ", #sum files: " + str(len(list_files)))

extract_trans(list_files, 'processed_trans.csv')

In [ ]:
# read contents of csv file
file = pd.read_csv("processed_trans.csv")
  
# adding header
headerList = ['sessionID', 'text']
  
# converting data frame to csv
file.to_csv("processed_trans_head.csv", header=headerList, index=False)

In [ ]:
list_category = [
                'ang',
                'hap',
                'sad',
                'neu',
                'fru',
                'exc',
                'fea',
                'sur',
                'dis',
                'oth',
                'xxx'
                ]

category = {}
for c_type in list_category:
    if category.__contains__(c_type):
        pass
    else:
        category[c_type] = len(category)

In [ ]:
def find_category(lines):
    is_target = True
    
    id = ''
    c_label = ''
    list_ret = []
    
    for line in lines:
        
        if is_target == True:
            
            try:
                id          = line.split('\t')[1].strip()  #  extract ID
                c_label  = line.split('\t')[2].strip()  #  extract category
                if not category.__contains__(c_label):
                    print("ERROR nokey ", c_label)
                    sys.exit()
                
                list_ret.append( [id, c_label] )
                is_target = False

            except:
                print("ERROR ", line)
                sys.exit()
        
        else:
            if line == '\n':
                is_target = True
        
    return list_ret

In [ ]:
def extract_labels(list_in_file, out_file) :
    id = ''
    lines = []
    list_ret = []
    
    for in_file in list_in_file:
        
        with open(in_file, 'r') as f1:
            lines = f1.readlines()
            lines = lines[2:]                           # remove head
            list_ret = find_category(lines)
            
        list_ret = sorted(list_ret)                   # sort based on first element
    
        with open(out_file, 'a') as f2:
            csv_writer = csv.writer(f2)
            csv_writer.writerows(list_ret)

In [ ]:
# [schema] ID, label [csv]

list_files = []
list_avoid_dir = ['Attribute', 'Categorical', 'Self-evaluation']

for x in range(5):
    sess_name = 'Session' + str(x+1)

    path = '/media/phamnhattruong/ReSe/Datasets/Emotion/IEMOCAP/IEMOCAP_full_release/IEMOCAP_full_release/' + sess_name + '/dialog/EmoEvaluation/'
    file_search(path, list_files, list_avoid_dir)
    list_files = sorted(list_files)

    print(sess_name + ", #sum files: " + str(len(list_files)))

extract_labels(list_files, "processed_labels.csv")

In [ ]:
# read contents of csv file
file = pd.read_csv("processed_labels.csv")
  
# adding header
headerList = ['sessionID', 'label']
  
# converting data frame to csv
file.to_csv("processed_labels_head.csv", header=headerList, index=False)

In [ ]:
dfl = pd.read_csv('processed_labels_head.csv')
dfl.loc[dfl["label"] == "ang", "label"] = 0
dfl.loc[dfl["label"] == "hap", "label"] = 1
dfl.loc[dfl["label"] == "exc", "label"] = 1
dfl.loc[dfl["label"] == "sad", "label"] = 2
dfl.loc[dfl["label"] == "neu", "label"] = 3
dfl.loc[dfl["label"] == "fru", "label"] = -1
dfl.loc[dfl["label"] == "fea", "label"] = -1
dfl.loc[dfl["label"] == "sur", "label"] = -1
dfl.loc[dfl["label"] == "dis", "label"] = -1
dfl.loc[dfl["label"] == "oth", "label"] = -1
dfl.loc[dfl["label"] == "xxx", "label"] = -1
dfl.head(10)

In [ ]:
dfl.to_csv("processed_digital_labels_head.csv", index=False)

In [ ]:
# reading two csv files
data1 = pd.read_csv('processed_trans_head.csv')
data2 = pd.read_csv('processed_digital_labels_head.csv')
  
# using merge function by setting how='inner'
translabels = pd.merge(data1, data2, 
                   on='sessionID', 
                   how='inner')

translabels.to_csv("processed_trans_labels_head.csv", index=False)

In [ ]:
list_files = []
for x in range(5):
    sess_name = 'Session' + str(x+1)
    path = '/media/phamnhattruong/ReSe/Datasets/Emotion/IEMOCAP/IEMOCAP_full_release/IEMOCAP_full_release/'+ sess_name + '/sentences/wav/'
    file_search(path, list_files)
    list_files = sorted(list_files)
    print (sess_name + ", #sum files: " + str(len(list_files)))

In [ ]:
# def audio2spectrogram(filepath):
#     #fig = plt.figure(figsize=(5,5))
#     samplerate, test_sound  = wavfile.read(filepath,mmap=True)
#     #print('samplerate',samplerate)
#     _, spectrogram = log_specgram(test_sound, samplerate)
#     #print(spectrogram.shape)
#     #print(type(spectrogram))
#     #plt.imshow(spectrogram.T, aspect='auto', origin='lower')
#     return spectrogram
    
# def audio2wave(filepath):
#     fig = plt.figure(figsize=(5,5))
#     samplerate, test_sound  = wavfile.read(filepath,mmap=True)
#     plt.plot(test_sound)

In [ ]:
# def log_specgram(audio, sample_rate, window_size=40,
#                  step_size=20, eps=1e-10):
#     nperseg = int(round(window_size * sample_rate / 1e3))
#     noverlap = int(round(step_size * sample_rate / 1e3))
#     #print('noverlap',noverlap)
#     #print('nperseg',nperseg)
#     freqs, _, spec = signal.spectrogram(audio,
#                                     fs=sample_rate,
#                                     window='hann',
#                                     nperseg=nperseg,
#                                     noverlap=noverlap,
#                                     detrend=False)
#     return freqs, np.log(spec.T.astype(np.float32) + eps)

In [ ]:
# # N_CHANNELS = 3
# def get_3d_spec(Sxx_in, moments=None):
#     if moments is not None:
#         (base_mean, base_std, delta_mean, delta_std,
#              delta2_mean, delta2_std) = moments
#     else:
#         base_mean, delta_mean, delta2_mean = (0, 0, 0)
#         base_std, delta_std, delta2_std = (1, 1, 1)
#     h, w = Sxx_in.shape
#     right1 = np.concatenate([Sxx_in[:, 0].reshape((h, -1)), Sxx_in], axis=1)[:, :-1]
#     delta = (Sxx_in - right1)[:, 1:]
#     delta_pad = delta[:, 0].reshape((h, -1))
#     delta = np.concatenate([delta_pad, delta], axis=1)
#     right2 = np.concatenate([delta[:, 0].reshape((h, -1)), delta], axis=1)[:, :-1]
#     delta2 = (delta - right2)[:, 1:]
#     delta2_pad = delta2[:, 0].reshape((h, -1))
#     delta2 = np.concatenate([delta2_pad, delta2], axis=1)
#     base = (Sxx_in - base_mean) / base_std
#     delta = (delta - delta_mean) / delta_std
#     delta2 = (delta2 - delta2_mean) / delta2_std
#     stacked = [arr.reshape((h, w, 1)) for arr in (base, delta, delta2)]
#     return np.concatenate(stacked, axis=2)

In [ ]:
df=pd.read_csv('processed_trans_labels_head.csv')
df.head()

In [ ]:
# df['label'] = df['label'].replace(['-1','0','1', '2', '3'],[-1,0,1,2,3])

In [ ]:
type(df['label'][0])

In [ ]:
# no_rows=len(list_files)
# # cnt = 0
# index=0
# sprectrogram_shape=[]
# docs = []
# bookmark=0
# extraLabel=0
# for everyFile in list_files:
#   if(everyFile.split('/')[-1].endswith('.wav')):
#     filename=everyFile.split('/')[-1].strip('.wav')
#     lable=df.loc[df['sessionID']==filename]['label'].values[0]
#     text=df.loc[df['sessionID']==filename]['text'].values[0]
#     # print('label',lable)
#     if(lable!=-1):
#       #sprectrogram_shape.append(audio2spectrogram(everyFile))
#       # spector=audio2spectrogram(everyFile)
#       # spector=get_3d_spec(spector)
#       # npimg = np.transpose(spector,(2,0,1))
#       # input_tensor=torch.tensor(npimg)
#       # input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
#       input_batch = vggish_input.wavfile_to_examples(everyFile)
#       # print(input_batch.size())
#       if (len(input_batch.size()) < 4) or (input_batch.size(dim=0) <= 1):
#         # print("Wrong", input_batch.size())
#         continue
#       #X, sample_rate = librosa.load(everyFile, res_type='kaiser_fast',sr=22050*2)
#       #sample_rate = np.array(sample_rate)
#       #mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=13),axis=0)
#       #feature = mfccs
#       elif (len(input_batch.size()) == 4) and (input_batch.size(dim=0) > 1):
#         # print("Correct", input_batch.size())
#         docs.append({
#            'fileName':everyFile.split('/')[-1].strip('.wav'),
#            'text':text,
#            'sprectrome':input_batch,
#            'label':lable
#                 })
#         index+=1
#         # print('index',index)
#         # cnt+=1
#         # if cnt > 100:
#           # break
#     else:
#       extraLabel=extraLabel+1
#       # print('extraLabel',extraLabel)

In [ ]:
# random.shuffle(docs)
# random.shuffle(docs)
# random.shuffle(docs)
# total_length=len(docs)
# train_length=int(.8*total_length)
# train_list=docs[0:train_length]
# test_list=docs[train_length:]
# print('no of items for train ',len(train_list))
# print('no of items for test ',len(test_list))
# # no of items for train  4424
# # no of items for test  1107

In [ ]:
# train_list[0]

In [ ]:
# train_file = open("train_data.pkl", "wb")

# pickle.dump(train_list, train_file)

# train_file.close()

# test_file = open("test_data.pkl", "wb")

# pickle.dump(test_list, test_file)

# test_file.close()

In [ ]:
train_file = open("train_data.pkl", "rb")

train_list = pickle.load(train_file)

print(train_list[0])

test_file = open("test_data.pkl", "rb")

test_list = pickle.load(test_file)

print(test_list[0])

In [ ]:
outputs_text= []
def hook_text(module, input, output):
    outputs_text.clear()
    outputs_text.append(output)
    return None

In [ ]:
outputs_audio= []
def hook_audio(module, input, output):
    outputs_audio.clear()
    outputs_audio.append(output)
    return None

In [ ]:
class MMSER(nn.Module):
    def __init__(self, num_classes=4):
        super(MMSER, self).__init__()
        self.num_classes=num_classes
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.text_model= BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes, return_dict=True)
        self.audio_model= vggish()
        
        
        self.text_model.bert.pooler.register_forward_hook(hook_text)
        # self.audio_model.features.register_forward_hook(hook_audio)
        
        for param in self.text_model.parameters():
          param.requires_grad = False
        # for param in self.audio_model.parameters():
        #   param.requires_grad = False

        self.dropout = nn.Dropout(.5)
        self.linear1 = nn.Linear(768, 128)
        self.linear2 = nn.Linear(256, num_classes)

        self.softmax = nn.Softmax(dim=1)
        
    def forward(self,text,audio):
        self.text_model(text)
        # self.audio_model(audio)
        # audio_embed=outputs_audio[0]
        audio_embed = self.audio_model(audio)
        print("A1", audio_embed.shape)
        text_embed=outputs_text[0]
        text_embed = self.linear1(text_embed)
        print("T", text_embed.shape)
        # audio_embed=torch.flatten(audio_embed, start_dim=2)#a1,a2,a3......al{a of dim c} 
        if (len(audio_embed.size()) == 1):
            audio_embed = torch.unsqueeze(audio_embed, dim=0)
        elif (audio_embed.shape[0] == 1) and (audio_embed.shape[1] == 128):
            audio_embed = audio_embed
        elif (audio_embed.shape[0] > 1) and (audio_embed.shape[1] == 128):
            audio_embed=torch.sum(audio_embed, dim=0)
            audio_embed = torch.unsqueeze(audio_embed, dim=0)
        print("A2", audio_embed.shape)
        concat_embded=torch.cat((text_embed,audio_embed),1)
        print("Concatenated", concat_embded.shape)
        x=self.dropout(concat_embded)
        x=self.linear2(x)
        return x

In [ ]:
model=MMSER(num_classes=4)
# model.to(device)

In [ ]:
# for name, param in model.named_parameters():
#       if(param.requires_grad):
#         print(name)
#       else:
#         print('no grad',name)

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# label1=train_list[100]['label']
# # label1 = int(label1)
# text=train_list[100]['text']
# input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# input_ids=input_ids.to(device)
# label1=torch.tensor([label1])
# label1=label1.to(device)
# sprectrome=train_list[100]['sprectrome']
# sprectrome=sprectrome.to(device)
# model.to(device)
# model.eval()
# with torch.no_grad():
#     output = model(input_ids,sprectrome)
#     #output.squeeze().shape
#     #output=torch.flatten(output, start_dim=2)
#     #print(output.shape)
#     #output=torch.sum(output, dim=2)
#     print(output)

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter(log_dir='logs/')

In [ ]:
total_steps = 1
NUM_EPOCHS=5
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.train()
model.to(device)
for epoch in range(NUM_EPOCHS):
  lr_scheduler.step()
  random.shuffle(train_list)
  for every_trainlist in train_list:
    label1=every_trainlist['label']
    label1 = int(label1)
    text=every_trainlist['text']
    label1=torch.tensor([label1])
    sprectrome=every_trainlist['sprectrome']
    if(sprectrome.shape[2]>65):
      optimizer.zero_grad()
      input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0) 
      sprectrome = sprectrome.to(device)
      label1=label1.to(device)
      input_ids=input_ids.to(device)
      output = model(input_ids,sprectrome)
      #print('softmax output ',output)
      loss = criterion(output, label1)
      #print('label1',label1)
      print('loss',loss.item())
      loss.backward()
      optimizer.step()
      _, preds = torch.max(output, 1)
      accuracy = torch.sum(preds == label1)
      print('accuracy.item()',accuracy.item())
      #print('preds',preds)
      if total_steps % 10 == 0:
        with torch.no_grad():
          _, preds = torch.max(output, 1)
          accuracy = torch.sum(preds == label1)
          #print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'.format(epoch + 1, total_steps, loss.item(), accuracy.item()))
          writer.add_scalar('loss', loss.item(), total_steps)
          writer.add_scalar('accuracy', accuracy.item(), total_steps)                     
      total_steps+=1

In [ ]:
# torch.save(model, 'vggish_bert.pt')

In [ ]:
# model=torch.load('vggish_bert.pt')

In [ ]:
y_actu=[]
y_pred=[]
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.to(device)
model.eval()
for every_test_list in test_list:
    label1=every_test_list['label']
    label1=torch.tensor([label1])
    label1 = label1.to(device)
    sprectrome=every_test_list['sprectrome']
    sprectrome = sprectrome.to(device)
    text=every_test_list['text']
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    input_ids = input_ids.to(device)
    with torch.no_grad():
      if(sprectrome.shape[2]>65):
        #sprectrome = sprectrome.to('cuda')
        #label1=label1.to('cuda')
        output = model(input_ids,sprectrome)
        _, preds = torch.max(output, 1)
        y_actu.append(label1.cpu().numpy()[0])
        y_pred.append(preds.cpu().numpy()[0])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actu, y_pred)

In [ ]:
# import urllib
# url, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)
embedding_model = vggish()
embedding_model.to(device)
embedding_model.eval()
example = vggish_input.wavfile_to_examples("bus_chatter.wav")
example = example.to(device)
embeddings = embedding_model.forward(example)

In [ ]:
embeddings.shape

In [ ]:
# audio_embed=torch.flatten(embeddings, start_dim=0)#a1,a2,a3......al{a of dim c} 
# audio_embed=torch.sum(embeddings, dim=0)

In [ ]:
# audio_embed.shape

In [ ]:
audio_embed = torch.empty(1, 128)
audio_embed.size()

In [ ]:
len(audio_embed.size())

In [ ]:
print("A1", audio_embed.shape)
if (len(audio_embed.size()) == 1):
    audio_embed = torch.unsqueeze(audio_embed, dim=0)
elif (audio_embed.shape[0] == 1) and (audio_embed.shape[1] == 128):
    audio_embed = audio_embed
elif (audio_embed.shape[0] > 1) and (audio_embed.shape[1] == 128):
    audio_embed=torch.sum(audio_embed, dim=0)
    audio_embed = torch.unsqueeze(audio_embed, dim=0)
print("A2", audio_embed.shape)

In [ ]:
class MMSER_DCCA(nn.Module):
    def __init__(self, num_classes=4):
        super(MMSER_DCCA, self).__init__()
        self.num_classes=num_classes
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.text_model= BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes, return_dict=True)
        self.audio_model= vggish()
        
        
        self.text_model.bert.pooler.register_forward_hook(hook_text)
        # self.audio_model.features.register_forward_hook(hook_audio)
        
        for param in self.text_model.parameters():
          param.requires_grad = False
        # for param in self.audio_model.parameters():
        #   param.requires_grad = False

        self.dropout = nn.Dropout(.5)
        self.linear1 = nn.Linear(768, 128)
        self.linear2 = nn.Linear(256, num_classes)

        self.softmax = nn.Softmax(dim=1)
        
    def forward(self,text,audio):
        self.text_model(text)
        # self.audio_model(audio)
        # audio_embed=outputs_audio[0]
        audio_embed = self.audio_model(audio)
        print("A1", audio_embed.shape)
        text_embed=outputs_text[0]
        text_embed = self.linear1(text_embed)
        print("T", text_embed.shape)
        # audio_embed=torch.flatten(audio_embed, start_dim=2)#a1,a2,a3......al{a of dim c} 
        if (len(audio_embed.size()) == 1):
            audio_embed = torch.unsqueeze(audio_embed, dim=0)
        elif (audio_embed.shape[0] == 1) and (audio_embed.shape[1] == 128):
            audio_embed = audio_embed
        elif (audio_embed.shape[0] > 1) and (audio_embed.shape[1] == 128):
            audio_embed=torch.sum(audio_embed, dim=0)
            audio_embed = torch.unsqueeze(audio_embed, dim=0)
        print("A2", audio_embed.shape)
        dcca = DCCA(latent_dims=2, encoders=[text_embed, audio_embed])
        return dcca

In [ ]:
dcca_model = MMSER_DCCA(num_classes=4)